In [1]:
from pprint import pprint
from typing import Callable
import librosa
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers.pipelines.automatic_speech_recognition import AutomaticSpeechRecognitionPipeline

torch.set_float32_matmul_precision("medium")

/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.set_float32_matmul_precision("medium")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, attn_implementation='flash_attention_2',
).eval().cuda()
# model.generation_config.cache_implementation = "static"
# model.generation_config.max_new_tokens = 324
# model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
processor = AutoProcessor.from_pretrained(model_id)
# pipe = AutomaticSpeechRecognitionPipeline(
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     torch_dtype=torch.float16,
#     stride_length_s=0.3,
#     chunk_length_s=10,
#     device=device,
# )

In [5]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)

/tmp/ipykernel_104711/1580981099.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)
/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/home/demontego/whisper/tests/test_data/output.wav'

In [ ]:
len(audio) / 16000

In [6]:
import numpy as np


def chunk_audio_with_overlap(
    audio: np.ndarray,  # (n_samples,)
    vad: torch.nn.Module,
    get_speech_timestamps: Callable,
    sr: int = 16000,
    chunk_sec: int = 10,
    overlap_sec: float = 0.5
) -> np.ndarray[np.ndarray[np.float16]]:  # (N_chunks, chunk_len)
    """
    Режет аудиомассив на куски с перекрытием и возвращает массив чанков.

    :param audio: 1D np.ndarray аудиосигнала (float32 или int16)
    :param sr: sample rate (по умолчанию 16kHz)
    :param chunk_sec: длина каждого чанка в секундах
    :param overlap_sec: длина перекрытия в секундах
    :return: np.ndarray формы (N_chunks, chunk_len) с dtype float32
    """
    if audio.ndim != 1:
        raise ValueError("Input audio must be a 1D np.ndarray.")

    # Приводим к float32
    audio = audio.astype(np.float16)

    chunk_len = int(sr * chunk_sec)
    step = int(chunk_len - sr * overlap_sec)
    total_len = len(audio)

    chunks = []
    seconds = []

    for start in range(0, total_len, step):
        end = start + chunk_len
        chunk = audio[start:end]
        if len(chunk) < chunk_len:
            chunk = np.pad(chunk, (0, chunk_len - len(chunk)))
        speech_timestamps = get_speech_timestamps(chunk, vad, return_seconds=True)
        sum_talk_seconds = sum(item['end'] - item['start'] for item in speech_timestamps)
        if speech_timestamps and sum_talk_seconds > 1.0:
            chunks.append(chunk)
            seconds.append(start / sr)

    return np.stack(chunks), seconds

In [ ]:
vad, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=False,
    onnx=True
)

In [6]:
chunked_audio, seconds = chunk_audio_with_overlap(audio, vad, utils[0])

NameError: name 'chunk_audio_with_overlap' is not defined

In [9]:
long_audio = np.vstack([chunked_audio] * 1)

In [ ]:
long_audio.shape

In [11]:
proccessed_audio = processor.feature_extractor(long_audio, sampling_rate=16000, return_tensors="pt").input_features.to(torch.float16).cuda()

In [ ]:
proccessed_audio.shape

In [14]:
results = model.generate(
    input_features=proccessed_audio,
    forced_decoder_ids=[[50258, 50360]],
    length_penalty=1.0,
    max_new_tokens=100,
    num_beams=3,
    condition_on_prev_tokens=True,
    # compression_ratio_threshold=1.35,
    temperature=0.1,
    return_dict_in_generate=True,
    return_timestamps=True)

In [40]:
def parse_whisper_output_proper(tokenizer_output, timestamps: list[float]) -> list[dict[str, float | str]]:
    """
    Правильный способ обработки через tokenizer Whisper
    """
    segments = []
    for tokens, timestamp in zip(tokenizer_output.numpy(), timestamps):
        # Используем встроенный метод декодирования
        # Этот метод уже есть в вашем процессоре
        decoded_segments, chunks = processor.tokenizer._decode_asr(
            [{"tokens": [tokens]}], 
            return_language=False,
            return_timestamps=True, 
            time_precision=0.02
        )
        for chunk in chunks['chunks']:
            segments.append({
                'start_time': chunk['timestamp'][0] + timestamp,
                'end_time': min(chunk['timestamp'][1], 10.0) + timestamp,
                'text': chunk['text'].strip()
            })
    
    return segments

In [ ]:
parse_whisper_output_proper(results, seconds)

In [45]:
results = processor.batch_decode(
    results,
    decode_with_timestamps=True,
    basic_normalize=True,
    skip_special_tokens=True,
    time_precision=0.02
)

In [44]:
import re

def parse_whisper_timestamps_simple(text: str) -> list[tuple[float, float, str]]:
    """
    Парсит простые случаи формата '<|start|> text <|end|>'
    """
    pattern = r'<\|(\d+\.?\d*)\|>\s*(.*?)\s*<\|(\d+\.?\d*)\|>'
    matches = re.findall(pattern, text)
    
    results = []
    for match in matches:
        start_time = float(match[0])
        end_time = float(match[2])
        content = match[1].strip()
        results.append((start_time, end_time, content))
    
    return results

In [ ]:
res

In [ ]:
decoded, chunks = processor.tokenizer._decode_asr(
        [{"tokens": [results.flatten().numpy()]}], 
        return_timestamps=True, 
        return_language=False, 
        time_precision=0.02
    )
chunks

In [ ]:
chunks

In [ ]:
result = pipe(audio, generate_kwargs=generate_kwargs, return_timestamps=True)
pprint(result)

In [ ]:
import librosa

from src.core.pipe import Pipe
from src.core.config import ASRConfig, AudioProcessorConfig

config_asr = ASRConfig(model_id='openai/whisper-large-v3-turbo')
config_processor = AudioProcessorConfig()
pipe = Pipe(config_asr, config_processor)

pipe.warmup()

In [5]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)

In [6]:
result = pipe([audio])[0]

In [ ]:
result